In [17]:
import dlib
import cv2
import numpy as np


In [10]:
model_path="/home/jal/.deepface/weights/vgg_face_weights.h5"
# Load the pre-trained face detection model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../raw_data/shape_predictor_81_face_landmarks.dat')


In [11]:
import os

In [12]:
os.getcwd()


'/home/jal/code/stan-kondratev/celebrities-identification-quest/notebooks'

In [20]:
folder_path = "../raw_data/actors/"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
file_list

['../raw_data/actors/Johnny Depp b&w (6).jpg',
 '../raw_data/actors/Tom Cruise b&w(18).jpg',
 '../raw_data/actors/Keanu Reeves b&w (11).jpg',
 '../raw_data/actors/Cate Blanchett B&w (2).jpg',
 '../raw_data/actors/Kate Winslet b&w(8).jpg',
 '../raw_data/actors/Johnny Depp b&w(5).jpg',
 '../raw_data/actors/Keanu Reeves b&w(10).jpg',
 '../raw_data/actors/Keanu Reeves b&w(12).jpg',
 '../raw_data/actors/Tom Cruise b&w (16).jpg',
 '../raw_data/actors/Johnny Depp b&w (4).jpg',
 '../raw_data/actors/Nicole Kidman b&w(13).jpg',
 '../raw_data/actors/Cate Blanchett b&w (1).jpg',
 '../raw_data/actors/Nicole Kidman b&w (15).jpg',
 '../raw_data/actors/Kate winslet b&w (7).jpg',
 '../raw_data/actors/Kate Winslet b&w (9).jpg',
 '../raw_data/actors/Nicole Kidman b&w (14).jpg',
 '../raw_data/actors/Tom Cruise b&w (17).jpg',
 '../raw_data/actors/Cate Blanchett b&w.jpg']

In [23]:

images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    image = cv2.imread(image_path+'.jpg')
    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the image
    faces = detector(gray)
    # We take only the first detected face, if exist
    if len(faces) == 0:
        continue
    face = faces[0]
    
    # We extract the landmarks   
    landmarks = predictor(image, face)
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

    # Calculate the center of the eyes
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)
    nose = landmarks[30]

    # Calculate the angle between the eyes and the horizontal line
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Rotate the image around the center of the face rectangle
    (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
    center = (x + w // 2, y + h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    if ((aligned_face.shape[0] == 0) or (aligned_face.shape[1] == 0) or (aligned_face.shape[2] == 0)):
        continue
    
    # Re apply the face crop on the aligned image
    gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
    faces_aligned = detector(gray)
    if not faces_aligned:
        continue
    x, y, w, h = faces_aligned[0].left(), faces_aligned[0].top(), faces_aligned[0].width(), faces_aligned[0].height()
    processed_face = aligned_face[y:y+h, x:x+w]
    if ((processed_face.shape[0] == 0) or (processed_face.shape[1] == 0) or (processed_face.shape[2] == 0)):
        continue
    
    # Save the final preprocessed image
    save_path = image_path+"_face_aligned.jpg"
    
    # black and white output:
    cv2.imwrite(save_path, processed_face) #cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(save_path, face_image)
    print(f"Face image saved: {save_path}")

Face image saved: ../raw_data/actors/Johnny Depp b&w (6)_face_aligned.jpg
Face image saved: ../raw_data/actors/Tom Cruise b&w(18)_face_aligned.jpg
Face image saved: ../raw_data/actors/Cate Blanchett B&w (2)_face_aligned.jpg
Face image saved: ../raw_data/actors/Kate Winslet b&w(8)_face_aligned.jpg
Face image saved: ../raw_data/actors/Johnny Depp b&w(5)_face_aligned.jpg
Face image saved: ../raw_data/actors/Keanu Reeves b&w(10)_face_aligned.jpg
Face image saved: ../raw_data/actors/Keanu Reeves b&w(12)_face_aligned.jpg
Face image saved: ../raw_data/actors/Tom Cruise b&w (16)_face_aligned.jpg
Face image saved: ../raw_data/actors/Johnny Depp b&w (4)_face_aligned.jpg
Face image saved: ../raw_data/actors/Nicole Kidman b&w(13)_face_aligned.jpg
Face image saved: ../raw_data/actors/Cate Blanchett b&w (1)_face_aligned.jpg
Face image saved: ../raw_data/actors/Nicole Kidman b&w (15)_face_aligned.jpg
Face image saved: ../raw_data/actors/Kate winslet b&w (7)_face_aligned.jpg
Face image saved: ../raw_d